In [2]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import transforms,datasets, models
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import os
import copy
import time

# Load Data

In [3]:
def loadData():
    data_transform = transforms.Compose([
            transforms.Resize(224), transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    trainDataset = datasets.ImageFolder(root='./All_61326/train_61326',
                                               transform=data_transform)
    trainLoader = torch.utils.data.DataLoader(trainDataset,
                                                 batch_size=1, shuffle=True,
                                                 num_workers=4)

    testDataset = datasets.ImageFolder(root='./All_61326/test_61326',
                                               transform=data_transform)
    testLoader = torch.utils.data.DataLoader(testDataset,
                                                 batch_size=1, shuffle=True,
                                                 num_workers=4)
    return trainLoader, testLoader

# Train The Model Using VGG Net Model

In [4]:
def trainTheModel(net, use_gpu, iterations, trainLoader, optimizer, criterion):
    iterations = iterations
    start = time.time()
    for epoch in range(iterations):
        print("Epoch:",epoch)
        epochStart = time.time()
        runningLoss = 0    
        net.train(True) # For training
        for data in trainLoader:
            print("Training Batch Data ......")
            inputs,labels = data
            # Wrap them in Variable
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), \
                    Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)  

            # Initialize gradients to zero
            optimizer.zero_grad()
            # Feed-forward input data through the network        
            outputs = net(inputs)        
            # Compute loss/error
            loss = criterion(F.log_softmax(outputs), labels)
            # Backpropagate loss and compute gradients
            loss.backward()
            # Update the network parameters
            optimizer.step()
            # Accumulate loss per batch
            runningLoss += loss.data[0]    
        avgTrainLoss = runningLoss/217.0
        epochEnd = time.time()-epochStart
        print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ;Time consumed: {:.0f}m {:.0f}s '\
              .format(epoch + 1,iterations,avgTrainLoss,epochEnd//60,epochEnd%60))
    end = time.time()-start
    print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))
    torch.save(net, 'vgg_model_3.ckpt')

# Test The Model

In [5]:
def testTheModel(path, testLoader):
    model = torch.load(path)
    model.train(False)
    running_correct = 0
    for data in testLoader:
        inputs,labels = data
        print("Input Label:",labels)
        # Wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs.cuda())
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predicted = predicted.cpu()
        else:
            inputs = Variable(inputs)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            print("Predicted Result:",predicted)
        running_correct += (predicted == labels).sum()
    avgTestAcc = running_correct/6.0
    print("Test Accuracy:",avgTestAcc)

In [9]:
# Size of train and test datasets
trainLoader, testLoader = loadData()
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

vggNet = models.vgg16()
# print(vggNet)

new_classifier = nn.Sequential(*list(vggNet.classifier.children())[:-1])
new_classifier.add_module('fc', nn.Linear(4096,6))
vggNet.classifier = new_classifier
print(vggNet)

init_weightConv1 = copy.deepcopy(vggNet.features[0].weight.data) # 1st conv layer
init_weightConv2 = copy.deepcopy(vggNet.features[2].weight.data) # 2nd conv layer

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')   
    vggNet = vggNet.cuda()

criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(vggNet.parameters(), lr=1e-4) # Adam

No. of samples in train set: 217
No. of samples in test set: 6
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_siz

In [6]:
trainTheModel(vggNet, use_gpu, 5, trainLoader, optimizer, criterion)

Epoch: 0
Training Batch Data ......


//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
T

Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
T

Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
T

Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
Training Batch Data ......
T

# Testing the test images accuracy

In [48]:
testTheModel('vgg_model_3.ckpt', testLoader)

Input Label: 
 2
[torch.LongTensor of size 1]

Predicted Result: 
 1
[torch.LongTensor of size 1]

Input Label: 
 4
[torch.LongTensor of size 1]

Predicted Result: 
 4
[torch.LongTensor of size 1]

Input Label: 
 3
[torch.LongTensor of size 1]

Predicted Result: 
 3
[torch.LongTensor of size 1]

Input Label: 
 0
[torch.LongTensor of size 1]

Predicted Result: 
 0
[torch.LongTensor of size 1]

Input Label: 
 5
[torch.LongTensor of size 1]

Predicted Result: 
 5
[torch.LongTensor of size 1]

Input Label: 
 1
[torch.LongTensor of size 1]

Predicted Result: 
 1
[torch.LongTensor of size 1]

Test Accuracy: 0.8333333333333334
